In [1]:
import requests
from bs4 import BeautifulSoup
import psycopg2

Read the webpage and get the HTML content

In [2]:
page = requests.get('http://eur-lex.europa.eu/LexUriServ/LexUriServ.do?uri=CELEX:31994R0040:en:HTML')

Parse the content with BeautifulSoup

In [5]:
soup = BeautifulSoup(page.text)

Extract CELEX number

In [4]:
CELEXN=soup.title.string.split("-")[2].replace(" ", "")

Now extracting information from the header: language, content, title and journal

In [7]:
metainfo=soup.head.find_all('meta')
languageinfo=metainfo[0]['content']
contentinfo=metainfo[2]['content']
titleinfo=metainfo[3]['content']
journalinfo=metainfo[5]['content'].split(";")[0]

In [8]:
print CELEXN
print languageinfo
print contentinfo
print titleinfo
print journalinfo

31994R0040
EN
Provisions under Article 235 EEC, Industrial and commercial property, Community law, European integration, 
Council Regulation (EC) No 40/94 of 20 December 1993 on the Community trade mark  
Official Journal L 011 , 14/01/1994 P. 0001 - 0036


Now looping in the main body to extract articles

In [9]:
bodyall=soup.body

Deinfe empty quantities to fill in the loop

In [152]:
articlenum = []
titleart = []
paragraphs = []
subparagraphs = []
title = []

Main function: loop in the body of the documents, identifies "title" if it starts with "TITLE", identify "articles" if it starts with "Article", extract the article number "articlenum" and extract the "paragraphs" for each article. For the moment the sub-paragraph (a),(b),(c) are concatenate within the sub-paragraph (need to find a better way)

In [153]:
for j,parg in enumerate(bodyall.find_all("p")):
    if parg.get_text()=="HAS ADOPTED THIS REGULATION:":
        print " -- HAS ADOPTED THIS REGULATION -- found, star loop across articles"
    if "TITLE" == parg.get_text().split(' ')[0]:
        tmp_title = parg.get_text().split(' ')[1]
    if "Article" == parg.get_text().split(' ')[0]:
        articlenum.append(int(filter(str.isdigit, str(parg.get_text()))))
        title.append(tmp_title)
        tmp_listp = []
        for i,sibling in  enumerate(parg.next_siblings):
            sib_txt = sibling.get_text()
            if i == 0: # If first, get the title
                titleart.append(sib_txt)
            elif sib_txt and sib_txt.split(' ')[0] != "SECTION":
                if sib_txt.split(' ')[0][0] == "(":
                    tmp_listp[-1] = tmp_listp[-1] + sib_txt
                else:
                    tmp_listp.append(sib_txt)
            elif not sib_txt: # If empty, break
                break
        paragraphs.append(tmp_listp)
        #subparagraphs.append(tmp_listsp)

 -- HAS ADOPTED THIS REGULATION -- found, star loop across articles


In [ ]:
print len(title) 
print len(paragraphs)
print len(titleart)
print len(articlenum)
print title[0] # This is in Roman numbers, can be converted in integer
print articlenum[0]
print titleart[0]
print paragraphs[0]

Creating a connection with the database for storing the information. I created a postgresSQL database that can be access using SQL language. In this case we use the package psycopg2 to access the database via python.

In [184]:
try:
    conn = psycopg2.connect("dbname='eu_law_db' user='postgres' host='localhost' password='1011'")
except:
    print "I am unable to connect to the database"

Create two tables in the database: The first table (eu_articles) contains a unique CELEX-article number combinations. Each row is an articles. The second table (eu_paragraph) contains all the paragraphs for all the articles. Each row is a paragraph, the unique key is the article number-unique paragraph number.

In [185]:
cur = conn.cursor()
cur.execute("CREATE TABLE eu_articles (celex varchar NOT NULL, title varchar NOT NULL, numarticle integer NOT NULL, titlearticle varchar NOT NULL, CONSTRAINT clex_numart PRIMARY KEY (celex,numarticle));")
cur.execute("CREATE TABLE eu_paragraph (celex varchar NOT NULL, numarticle integer NOT NULL, numpar SERIAL, paragraph varchar NOT NULL, CONSTRAINT clex_numart_numpar PRIMARY KEY (celex,numarticle,numpar));")
conn.commit()

In [189]:
eu_articles_imp=zip([CELEXN]*len(articlenum),title, articlenum, titleart)
eu_paragraph_imp=[]
for n,i in enumerate(paragraphs):
    for k in i:
        eu_paragraph_imp.append((CELEXN,articlenum[n],k))

Inserting all the information obtained in the tables

In [190]:
cur = conn.cursor()
cur.executemany("""INSERT INTO eu_articles (celex, title, numarticle, titlearticle) VALUES (%s,%s,%s,%s)""", eu_articles_imp)
conn.commit()

In [191]:
cur = conn.cursor()
cur.executemany("""INSERT INTO eu_paragraph (celex, numarticle, paragraph) VALUES (%s,%s,%s)""", eu_paragraph_imp)
conn.commit()

We start with listing all the .pdf in the folder (NOT USED RIGHT NOW)

In [ ]:
maindir="/Users/andrea/Documents/Work/eu_law_db"
datadir=maindir + "/data"
for i in os.listdir(datadir):
    print i